# Import libraries for interactively working in the notebook

In [2]:
#Base modules
import sys
import os
from datetime import datetime as dati

#Basic 3rd party packages
import h5py
import numpy as np
import json

#GUI elements
import tkinter as Tk
from tkinter import filedialog

#For working with images
from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar, SI_LENGTH_RECIPROCAL

#for generating some random numbers
import random

In [3]:
%matplotlib notebook

# My in-out module for a few operations on Velox emd

In [4]:
import modules.io as io
import importlib


In [13]:
importlib.reload(io)

<module 'modules.io' from '/Users/nielscautaerts/Documents/Projects/BigMax/3_Software/VeloxMeta/TEMMETA/modules/io.py'>

# Creating dummy hdf5 file

In [ ]:
with h5py.File("example.hdf5", "w") as nf:
    g1 = nf.create_group("Group1")
    g2 = nf.create_group("Group2/NestedGroup1")
    size = (100, 100)
    data = np.random.rand(*size)
    ds1 = g1.create_dataset("RandomImage", data = data)
    ds2 = g2.create_dataset("RandomThing", data = "Random string")
    #add some attributes
    g1.attrs["Attribute1"] = "Whatwhat"
    g1.attrs["Listatr"] = [0, 1, 2, 4]
    g2.attrs["Bla"] = 2
    g2.attrs["hmm"] = "Boo"
    ds1.attrs["What"] = "This is a dataset"
    ds2.attrs["teehee"] = "What is this"

# Visual file opening

In [6]:
f = io.open_emd_gui()

# CLI file opening

In [5]:
filename = os.path.expanduser("~/ownCloud/emdData&Reader/TestData/April2019/1520 EDS-HAADF 20190410.emd")

In [6]:
f = h5py.File(filename, 'r')

# Inspection of HDF5 node

In [7]:
help(io.scan_hdf5_node)

Help on function scan_hdf5_node in module modules.io:

scan_hdf5_node(hdf5_node, recursive: bool = True, full_path: bool = False, see_info: bool = True, tab_step: int = 4)
    Print the structure of an HDF5 node (can be root).
    
    Args:
        hdf5_node : The node to be investigated
        recursive (bool): if true it traverses all subgroups. If not it only prints top level of the current node.
        full_path (bool): print the entire group path of groups and datasets or only its name
        see_info (bool): print the array size and datatype next to datasets.
        tab_step (int): how many spaces to indent each level



In [8]:
io.scan_hdf5_node(f)

/
    Application
        Velox
             - DisplayLayout  ((1,), object)
    Data
        Image
            1c887212b79849869d14fd2b03db2481
                 - Data  ((512, 512, 1), float32)
                 - FrameLookupTable  ((1,), uint32)
                 - Metadata  ((60000, 1), uint8)
            515e962de04c4529b1e83a6c7ed565c7
                 - Data  ((512, 512, 1), float32)
                 - FrameLookupTable  ((1,), uint32)
                 - Metadata  ((60000, 1), uint8)
            84c474d76cea419399ee5eac5d9838ac
                 - Data  ((512, 512, 239), uint16)
                 - FrameLookupTable  ((239,), uint32)
                 - Metadata  ((60000, 239), uint8)
            ae260c6bc7ff47898f3321228d18ddbd
                 - Data  ((512, 512, 1), float32)
                 - FrameLookupTable  ((1,), uint32)
                 - Metadata  ((60000, 1), uint8)
        Line
             - fde442a54d7d4654b6386a1657c3629c  ((1,), object)
        Spectrum
            ac8fa

In [9]:
io.scan_hdf5_node(f["Operations"], full_path = True, recursive = False)

/Operations
    /Operations/DisplayLevelsOperation
    /Operations/EDSInputOperation
    /Operations/ImageQuantificationOperation
    /Operations/IntensityProfileOperation
    /Operations/MixOperation
     - /Operations/Operations  ((1,), object)
    /Operations/StemInputOperation


# Dealing with metadata

In [14]:
help(io.get_det_uuid)
help(io.get_meta_dict)
help(io.get_meta_dict_det_no)
help(io.print_pretty)
help(io.write_meta_json)    
help(io.read_meta_json)

Help on function get_det_uuid in module modules.io:

get_det_uuid(f: h5py._hl.files.File, sig: str, det_no: int)
    Get the UUID key for the detector signal based on a detector index number
    
    Args:
        f (h5py._hl.files.File): the HDF5 file opened with h5py.File
        sig (str):     Provide signal type (Data subgroup) as a string
                       ["Image", "Line", "Spectrum", "SpectrumImage", "Spectrumstream"]
        det_no (int):  The detector number, since they have UUID names. If not provided, turned into 0.
    
    Returns:
        str: the uuid that can be used as a key to access data and metadate

Help on function get_meta_dict in module modules.io:

get_meta_dict(f: h5py._hl.files.File, sig: str, det: str, frame: int = 0) -> dict
    General importing function of EMD metadata.
    
    Args:
        f (h5py._hl.files.File): the HDF5 file opened with h5py.File
        sig (str):     Provide signal type (Data subgroup) as a string
                       ["Ima

## Examples

In [119]:
meta_test = io.get_meta_dict_det_no(f, "Image", det_no=0, frame = 0)
meta_test2 = io.get_meta_dict_det_no(f, "Image", det_no=2, frame = 0)

In [17]:
io.print_pretty(meta_test)

{
    "Acquisition": {
        "AcquisitionDatetime": {
            "DateTime": "0"
        },
        "AcquisitionStartDatetime": {
            "DateTime": "1554902425"
        },
        "BeamType": "",
        "SourceType": "Monochromator"
    },
    "BinaryResult": {
        "AcquisitionUnit": "",
        "CompositionType": "",
        "Detector": "SuperXG1",
        "Encoding": "",
        "Offset": {
            "x": "-9.016565482561257e-009",
            "y": "-9.016565482561257e-009"
        },
        "PixelSize": {
            "height": "3.522095891625491e-011",
            "width": "3.522095891625491e-011"
        },
        "PixelUnitX": "m",
        "PixelUnitY": "m"
    },
    "Core": {
        "MetadataDefinitionVersion": "7.9",
        "MetadataSchemaVersion": "v1/2013/07",
        "guid": "00000000000000000000000000000000"
    },
    "CustomProperties": {
        "Aperture[C1].Name": {
            "type": "string",
            "value": "2000"
        },
        "Apertu

In [121]:
print(meta_test["Core"]["MetadataDefinitionVersion"])

7.9


In [21]:
#accessing and converting the data in the metadata
timestamp = int(meta_test["Acquisition"]["AcquisitionStartDatetime"]["DateTime"])
print("Timestamp: ", timestamp)
starttime = dati.fromtimestamp(timestamp)
print("DateTime: ", starttime)

Timestamp:  1554902425
DateTime:  2019-04-10 15:20:25


# Working with Data

## Data parsers

In [10]:
def get_data(f: h5py._hl.files.File, sig: str, det: str):
    """
    Returns EMD data using a signal string and detector uuid
    
    Args:
        f (h5py._hl.files.File): the HDF5 file opened with h5py.File
        sig (str):     Provide signal type (Data subgroup) as a string 
                       ["Image", "Line", "Spectrum", "SpectrumImage", "Spectrumstream"]
        det (int):  The detector index, which is translated to a UUID str.
        
    Returns:
        h5py._hl.dataset.Dataset
    """
    data=f['Data'][sig][det]['Data']
    return data

def get_image_data_det_no(f: h5py._hl.files.File, det_no: int):
    """
    Wrapper for importing EMD image data using a detector index.
    
    Args:
        f (h5py._hl.files.File): the HDF5 file opened with h5py.File
        det_no (int):  The detector index, which is translated to a UUID str.
        
    Returns:
        h5py._hl.dataset.Dataset
    """
    sig = "Image"
    det = get_det_uuid(f, sig, det_no)
    return get_data(f, sig, det)
    

    

In [199]:
'snIONnw.sdfnp'.lower()

'snionnw.sdfnp'

In [196]:
get_det_uuid = io.get_det_uuid
get_data = io.get_data
get_meta_dict = io.get_meta_dict


        


    




    



In [191]:
d[3:800000].flatten()

array([65535, 65535, 65535, ..., 65535, 65535, 65535], dtype=uint16)

In [124]:
duuid = get_det_uuid(f, "SpectrumStream", 0)
get_spectrum_stream_acqset(f, duuid)

{'encoding': 'uint16',
 'bincount': '4096',
 'StreamEncoding': 'uint16',
 'Size': '1048576',
 'RasterScanDefinition': {'Width': '512', 'Height': '512'}}

In [143]:
#d, md1, md2, flut = get_spectrum_stream_frames(f, 0)
allframes = get_spectrum_stream_frames(f, one_matrix = True)

'get_spectrum_stream_frames'  18258.94 ms


In [146]:
sys.getsizeof(allframes)/1000000

41.943168

In [197]:
someframes = get_spectrum_stream_frames(f, frames = [0, 1,2,4], one_matrix = True)

'get_frame_limits'  0.00 ms
'get_frame_limits'  0.01 ms
'get_frame_limits'  0.00 ms
'get_frame_limits'  0.00 ms
'convert_stream_to_sparse'  408.26 ms
'get_spectrum_stream_frames'  591.13 ms


In [198]:
someframes_list = get_spectrum_stream_frames(f, frames = [0, 1,2,4], one_matrix = False)

'get_frame_limits'  0.00 ms
'get_frame_limits'  0.01 ms
'get_frame_limits'  0.01 ms
'get_frame_limits'  0.02 ms
'convert_stream_to_sparse'  245.95 ms'convert_stream_to_sparse'  251.90 ms
'translate_stream_frame'  253.37 ms
'convert_stream_to_sparse'  261.25 ms
'translate_stream_frame'  264.01 ms

'translate_stream_frame'  280.64 ms
'convert_stream_to_sparse'  240.86 ms
'translate_stream_frame'  310.45 ms
'get_spectrum_stream_frames'  487.51 ms


In [170]:
allframes

<62652416x4096 sparse matrix of type '<class 'numpy.int16'>'
	with 740516 stored elements in Dictionary Of Keys format>

In [169]:
someframes

<1048576x4096 sparse matrix of type '<class 'numpy.int16'>'
	with 12488 stored elements in Dictionary Of Keys format>

In [202]:
mss = io.get_meta_dict_det_no(f, "SpectrumStream")
io.print_pretty(mss)

{
    "Acquisition": {
        "AcquisitionDatetime": {
            "DateTime": "0"
        },
        "AcquisitionStartDatetime": {
            "DateTime": "1554902425"
        },
        "BeamType": "",
        "SourceType": "Monochromator"
    },
    "BinaryResult": {
        "AcquisitionUnit": "",
        "CompositionType": "",
        "Detector": "SuperXG1",
        "Encoding": "",
        "Offset": {
            "x": "-9.016565482561257e-009",
            "y": "-9.016565482561257e-009"
        },
        "PixelSize": {
            "height": "3.522095891625491e-011",
            "width": "3.522095891625491e-011"
        },
        "PixelUnitX": "m",
        "PixelUnitY": "m"
    },
    "Core": {
        "MetadataDefinitionVersion": "7.9",
        "MetadataSchemaVersion": "v1/2013/07",
        "guid": "00000000000000000000000000000000"
    },
    "CustomProperties": {
        "Aperture[C1].Name": {
            "type": "string",
            "value": "2000"
        },
        "Apertu

In [203]:
lst = [0,3,5,7,8,5,3,5,4,5,6,7,8,3,5,4]
[(i,i+1) for i in range(5)]

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]

In [113]:
x, y = get_xy_from_inx(np.arange(512*512), 512, 512)

In [41]:
io.print_pretty(md)

{
    "Acquisition": {
        "AcquisitionDatetime": {
            "DateTime": "0"
        },
        "AcquisitionStartDatetime": {
            "DateTime": "1554902425"
        },
        "BeamType": "",
        "SourceType": "Monochromator"
    },
    "BinaryResult": {
        "AcquisitionUnit": "",
        "CompositionType": "",
        "Detector": "SuperXG1",
        "Encoding": "",
        "Offset": {
            "x": "-9.016565482561257e-009",
            "y": "-9.016565482561257e-009"
        },
        "PixelSize": {
            "height": "3.522095891625491e-011",
            "width": "3.522095891625491e-011"
        },
        "PixelUnitX": "m",
        "PixelUnitY": "m"
    },
    "Core": {
        "MetadataDefinitionVersion": "7.9",
        "MetadataSchemaVersion": "v1/2013/07",
        "guid": "00000000000000000000000000000000"
    },
    "CustomProperties": {
        "Aperture[C1].Name": {
            "type": "string",
            "value": "2000"
        },
        "Apertu

## Data plotters and exporters

#### Images

In [ ]:
def save_single_image(imgdata: np.ndarray, filename:str , metadata: dict, 
               scale_bar: bool = True, show_fig: bool = False, dpi: int = 100, save_meta: bool = True,
               sb_settings = {"location":'lower right', "color" : 'k', "length_fraction" : 0.15}, **kwargs):
    
    #initialize the figure and axes objects
    fig = plt.figure(frameon=False, figsize = (imgdata.shape[0]/dpi, imgdata.shape[1]/dpi))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    #plot the figure on the axes
    s = ax.imshow(imdat, **kwargs)
    
    if scale_bar:
        #get scale bar info from metadata
        px=(float(metadata["BinaryResult"]["PixelSize"]["width"]))
        unit=metadata["BinaryResult"]["PixelUnitX"]
        #check the units and adjust sb accordingly
        if unit=='1/m':
            px=px*10**(-9)
            scalebar = ScaleBar(px, '1/nm', SI_LENGTH_RECIPROCAL, **sb_settings)
        else:
            scalebar = ScaleBar(px, unit, **sb_settings)
        plt.gca().add_artist(scalebar)
    #save the figure
    plt.savefig(filename, dpi = dpi)
    
    if show_fig:
        plt.show()
    else:
        plt.close()
    
    if save_meta:
        #if metadata save the metadata to a json file with the same name
        path, ext = os.path.splitext(filename)
        write_meta_json(path+".json", metadata)
    
    

In [ ]:
imdat = get_image_data_det_no(f, 0)[:,:,0]
metdat = get_meta_dict_det_no(f, "Image", 0)
save_single_image(imdat, "testfigure.tiff", metdat, save_meta = True, cmap = "plasma")

In [ ]:
#does not work
def load_image_tiff(filename: str, load_meta: bool = True):
    with Image.open(filename) as tif:
        data = tif
        metadata = None
        if load_meta:
            path, ext = os.path.splitext(filename)
            metadata = read_meta_json(path+".json")
        return data, metadata
        #except:
        #    return data

In [ ]:
ima, met = load_image_tiff("testfigure.tiff")

#### Spectra

# Hyperspy parsers

In [ ]:
#Higher level 3rd party packages
import numba as nb
import hyperspy.api as hs


In [ ]:
fhs = hs.load(filename)

In [ ]:
fhs

In [ ]:
#hyperspy interpreted metadata
help(type(fhs[5]))

In [ ]:
fhs[5].original_metadata.Acquisition